In [2]:
import os
from itertools import chain
from functools import partial

import numpy as np
import scipy as sp
from scipy import sparse
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix

from imblearn.ensemble import EasyEnsembleClassifier

In [3]:
data_path = "/afs/cern.ch/user/a/ananiev/cernbox/output/"

In [4]:
data = pd.read_csv(os.path.join(data_path, "hgg_features.tsv"), sep="\t")

In [5]:
RANDOM_STATE = 23

## Descriptive statistics

Count number of entries per class

In [89]:
def weight_descriptive(data):
    res = {}
    
    event_counts = data["label"].value_counts().sort_values(ascending=False)
    res["event_num"] = event_counts
    
    weight_sums = data.groupby("label")["weight"].sum().sort_values(ascending=False)
    res["weight_sums"] = weight_sums
    
    weight_frac = weight_sums/data["weight"].sum()
    res["weight_frac"] = weight_frac
    
    weight_per_event = weight_sums/event_counts
    res["weight_per_event"] = weight_per_event
    
    return pd.DataFrame(res)
weight_descriptive(data)

,event_num,weight_sums,weight_frac,weight_per_event
VBF,10369,9.612055e-05,0.064293,9.269992e-09
Wp,2112,1.702021e-05,0.011384,8.058813e-09
Z,4571,1.697019e-05,0.011351,3.712578e-09
gg,26258,1.364896e-03,0.912952,5.198020e-08
tt,9861,2.851057e-08,0.000019,2.891245e-12


To summarize:

* **tt** events are not a lot, and they have relatively small weight
* **Wp**, **Z** are quite few, but the weight is significant
* **VBF** are quite a lot and the weight is mediocre
* **gg** are the decent majority, while their weight is quite small

In [10]:
data.columns

Index(['photon_n', 'photon_1lead_pt', 'photon_1lead_eta', 'photon_1lead_phi',
       'photon_1lead_E', 'photon_1lead_etcone20', 'photon_2lead_pt',
       'photon_2lead_eta', 'photon_2lead_phi', 'photon_2lead_E',
       'photon_2lead_etcone20', 'h_mass', 'lep_n', 'lep_pt_min', 'lep_pt_max',
       'lep_pt_mean', 'lep_pt_sum', 'lep_pt_std', 'lep_phi_min', 'lep_phi_max',
       'lep_phi_mean', 'lep_phi_sum', 'lep_phi_std', 'lep_E_min', 'lep_E_max',
       'lep_E_mean', 'lep_E_sum', 'lep_E_std', 'lep_theta_min',
       'lep_theta_max', 'lep_theta_mean', 'lep_theta_sum', 'lep_theta_std',
       'lep_charge_min', 'lep_charge_max', 'lep_charge_mean', 'lep_charge_sum',
       'lep_charge_std', 'lep_z0_min', 'lep_z0_max', 'lep_z0_mean',
       'lep_z0_sum', 'lep_z0_std', 'lep_ptcone30_min', 'lep_ptcone30_max',
       'lep_ptcone30_mean', 'lep_ptcone30_sum', 'lep_ptcone30_std',
       'lep_etcone20_min', 'lep_etcone20_max', 'lep_etcone20_mean',
       'lep_etcone20_sum', 'lep_etcone20_std', 'wei

Let's train baseline model with ensemble of trees in order to identify at least somewhat relevant features to the analysis

In [11]:
def train_test_val_split(dataset, target, test_size, val_size, *args, **kwargs):
    shuffle = kwargs.get("shuffle", True)
    del kwargs["shuffle"]
    X_train_val, X_test, y_train_val, y_test = train_test_split(data.drop(columns=[target]), data[target], *args, test_size=test_size, shuffle=shuffle, **kwargs)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_size, shuffle=False, **kwargs)
    return (pd.concat([X_train, y_train], axis="columns"),
            pd.concat([X_val, y_val], axis="columns"),
            pd.concat([X_test, y_test], axis="columns"))

In [12]:
d_train, d_val, d_test = train_test_val_split(data, "label", 0.2, 0.1, shuffle=True, random_state=RANDOM_STATE)

In [13]:
class KeepFeatures(TransformerMixin):
    def __init__(self, features):
        self.features = features
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, y=None, **fit_params):
        return X[self.features].copy()

In [14]:
class FillNa(TransformerMixin):
    def __init__(self, mapping):
        if isinstance(mapping, dict):
            self.mapping = mapping
            self.value = None
        else:
            self.value = mapping
            self.mapping = None
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, y=None, **fit_params):
        res = X.copy()
        if self.mapping is not None:
            for col, val in self.mapping.items():
                res[col].fillna(val, inplace=True)
        if self.value is not None:
            res.fillna(self.value, inplace=True)
        return res

In [15]:
def pipeline_maker(pipeline, *args,  **kwargs):
    def instantiate():
        return pipeline(*args, **kwargs)
    return instantiate

Initially we choose features describing the event in general (e.g. photon number, lepton number, missing energy), then, since two photons are important features of the event we pick two the most energetic photons and include all the features describing them in detail (e.g. eta, phi, pt, E). Then we include aggregated features for leptons, like sum of phi, sum of energies, sum of theta. Finally, we include `h_mass`.

Sometimes there are no leptons in the event, then we impose zero values for all the features describing them.

Find the full feature list below.

In [16]:
BaselineFeaturePipeline = \
    pipeline_maker(Pipeline, steps=[
        ('keep_features', KeepFeatures(['photon_n', 'photon_1lead_pt',
                                        'photon_1lead_eta', 'photon_1lead_phi',
                                        'photon_1lead_E', 'photon_1lead_etcone20',
                                        'photon_2lead_pt', 'photon_2lead_eta',
                                        'photon_2lead_phi', 'photon_2lead_E',
                                        'photon_2lead_etcone20', 'h_mass',
                                        'met_et', 'met_phi', 'lep_pt_sum',
                                        'lep_phi_sum', 'lep_E_sum', 'lep_theta_sum',
                                        'lep_charge_sum', 'lep_ptcone30_sum',
                                        'lep_etcone20_sum', 'label', 'weight'])),
        ('filna', FillNa(0.))
    ])
baseline_feature_pipeline = BaselineFeaturePipeline().fit(d_train)
baseline_features = baseline_feature_pipeline.transform(d_train)
baseline_features

,photon_n,photon_1lead_pt,photon_1lead_eta,photon_1lead_phi,photon_1lead_E,photon_1lead_etcone20,photon_2lead_pt,photon_2lead_eta,photon_2lead_phi,photon_2lead_E,...,met_phi,lep_pt_sum,lep_phi_sum,lep_E_sum,lep_theta_sum,lep_charge_sum,lep_ptcone30_sum,lep_etcone20_sum,label,weight
34762,2,68985.805,0.673179,-2.755656,85216.230,-33.086610,51873.760,-0.020217,0.261481,51884.363,...,-0.911852,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,gg,6.426950e-08
17522,2,65926.590,0.837155,-0.814143,90409.340,-849.875100,53449.170,-0.161098,1.488379,54144.246,...,0.773666,90692.882812,2.059564,98187.476562,-1.300542,1.0,1044.006714,4777.929199,tt,-6.256355e-12
47589,2,66244.650,0.284815,-0.899317,68949.734,-430.340760,49499.246,-0.664182,2.492585,60824.562,...,0.882567,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,gg,6.509352e-08
1178,2,77618.510,1.311288,0.615234,154477.250,-1098.243200,40889.438,-0.005911,2.722577,40890.152,...,-2.790231,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,Wp,9.925904e-09
44652,2,62168.310,-1.100176,-2.316379,103743.570,-257.241940,43533.900,0.573396,2.214298,50888.734,...,-0.120469,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,gg,6.574681e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27005,2,112069.280,-0.583809,-2.605017,131716.360,-653.104550,28948.701,0.584124,1.303498,34029.387,...,0.420166,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,tt,6.959701e-12
3731,2,54486.168,0.245120,-2.648435,56131.250,-50.511497,48930.305,-1.057293,0.539349,78923.680,...,-2.251780,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,Z,4.961111e-09
19695,2,62460.710,0.404643,-2.093862,67644.390,-138.293850,48761.650,-0.666147,1.519702,59986.727,...,0.057860,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,gg,3.565212e-08
40489,2,66830.560,0.162774,-1.102934,67717.870,-623.463500,45487.150,-0.942891,1.962060,67250.305,...,-1.235665,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,gg,6.799428e-08


In [17]:
def split_targets(data, targets, drop=None):
    flat_target = False
    if not isinstance(targets, list):
        flat_target = True
    to_drop = drop if drop is not None else []
    to_drop += targets if not flat_target else [targets]
    
    Xs = data.drop(columns=to_drop)
    ys = data[targets]
    
    return Xs, ys

In [18]:
baseline_model = RandomForestClassifier(random_state=RANDOM_STATE).fit(*split_targets(baseline_features, "label", drop=["weight"]), sample_weight=baseline_features["weight"])

In [19]:
def extract_importances(feature_importances, feature_names):
    return pd.DataFrame(zip(feature_names, feature_importances),
                                      columns=["feature", "score"]) \
             .set_index("feature") \
             .sort_values("score", axis="index", ascending=False)

In [20]:
def get_ensemble_feature_importances(ensemble_model):
    importances = []
    for est in ensemble_model.estimators_:
        importances.append(est.steps[-1][1].feature_importances_)
    importances = np.vstack(importances)
    return np.median(importances, axis=0)

In [66]:
def report_importances(model, X_test):
    if hasattr(model, "feature_importances_"):
        feature_importances = model.feature_importances_
    elif hasattr(model, "estimators_") and hasattr(model.estimators_[0].steps[-1][1], "feature_importances_"):
        feature_importances = get_ensemble_feature_importances(model)
    else:
        print("Model doesn't seem to provide feature importances. Skip feature importances report")
        return
    nice_importances = extract_importances(feature_importances, X_test.columns)
    print("Feature importances")
    display(nice_importances)

In [81]:
def report_class_summary(model, feature_pipeline, X_test, y_test, weights=None):
    if not hasattr(model, "classes_"):
        print("Model doesn't seem to be a classificator. Skip classification summary report")
        return
    classes = model.classes_
    labeler = feature_pipeline.named_steps.get("labels")
    if labeler is not None:
        classes = labeler.transformer.inverse_transform(classes)
    print("Model classes")
    display(classes)
    pred = model.predict(X_test)
    conf = confusion_matrix(y_test, pred)
    print("Confusion matrix, normalized")
    display(conf)
    conf = confusion_matrix(y_test, pred, sample_weight=weights)
    print("-Log confusion matrix of weights")
    display(-np.log(conf))

In [76]:
def report_descriptive(X_test, y_test, weights=None):
    data = pd.concat([X_test, y_test], axis="columns")
    if weights is not None:
        data = data.assign(weight=weights)
        
    if "weight" in X_test.columns:
        weight_sums = data.groupby("label")["weight"].sum()
        print("Sum of weights")
        display(weight_sums)

In [77]:
def report(model, feature_pipeline, d_test, drop=None):
    X_test, y_test = split_targets(d_test, "label", drop=drop)
    weights = d_test["weight"]
    report_descriptive(X_test, y_test, weights=weights)
    report_class_summary(model, feature_pipeline, X_test, y_test, weights=weights)
    report_importances(model, X_test)

After the model has been trained, let's see the report describing the baseline.

We have 5 classes and a confusion matrix for them.

We also provide confusion matrix for weights in somewhat sophisticated format. Due to weights can differ by orders of magnitude, it is easier to compare logarithms. Since all weights are < 1, it is convenient to remove minus sign close to numbers in the matrix. As a result, the smaller number on the diagonal of the matrix the better.

Finally we show feature importances coming from decision tree classifier.

In [78]:
report(baseline_model, baseline_feature_pipeline, baseline_feature_pipeline.transform(d_test), drop=["weight"])

Model classes


array(['VBF', 'Wp', 'Z', 'gg', 'tt'], dtype=object)

Confusion matrix, normalized


array([[ 717,    3,    8, 1304,   22],
       [ 128,   20,   10,  236,   58],
       [ 235,    4,   58,  603,   74],
       [ 620,    3,   16, 4540,   25],
       [ 472,   43,   40,  695,  701]])

-Log confusion matrix of weights


array([[11.93201359, 17.21032844, 16.55685393, 11.31630401, 15.50961559],
       [13.75567166, 15.58087654, 16.39603933, 13.1824441 , 14.66865602],
       [13.99952044, 18.33243011, 15.26140177, 12.99071183, 15.1836116 ],
       [10.34108454, 15.53840086, 13.97303262,  8.35967645, 13.71694805],
       [20.34227256, 23.37487264, 23.2164997 , 19.98716491, 20.05360614]])

Feature importances


,score
feature,
photon_1lead_pt,0.092880
photon_1lead_E,0.082657
met_et,0.076067
photon_2lead_pt,0.074722
photon_2lead_eta,0.074145
h_mass,0.072481
photon_1lead_eta,0.071895
photon_2lead_E,0.071594
photon_2lead_etcone20,0.071456


Apparently `gg` events are so many, that other events become classified as `gg` just statistically. This means that the dataset is imbalanced, and we should find some approach to target this issue.

We also choose subset of features to work with. We pick only significantly relevant features. See the list of them below.

In [42]:
def mask_by_levels(data, threshold=None, limit=None):
    thr_mask = np.array(True)
    if threshold is not None:
        if isinstance(threshold, dict):
            thr_mask = np.all([data[k] > v for k,v in threshold.items()])
        else:
            thr_mask = np.all(data > threshold, axis=1)
    
    lim_mask = np.array(True)
    if limit is not None:
        if isinstance(limit, dict):
            lim_mask = np.all([data[k] < v for k,v in limit.items()])
        else:
            lim_mask = np.all(data < limit, axis=1)
            
    mask = thr_mask & lim_mask
    return mask

In [43]:
def filter_by_levels(data, threshold=None, limit=None):
    mask = mask_by_levels(data, threshold=threshold, limit=limit)
    
    if np.all(mask):
        return data.copy()
    else:
        return data[mask].copy()

In [45]:
subcolumns = list(filter_by_levels(
                extract_importances(
                    baseline_model.feature_importances_,
                    split_targets(baseline_features, "label", drop=["weight"])[0].columns
                ),
                threshold=0.05
             ).index)
subcolumns

['photon_1lead_pt',
 'photon_1lead_E',
 'met_et',
 'photon_2lead_pt',
 'photon_2lead_eta',
 'h_mass',
 'photon_1lead_eta',
 'photon_2lead_E',
 'photon_2lead_etcone20',
 'photon_1lead_phi',
 'photon_2lead_phi',
 'met_phi',
 'photon_1lead_etcone20']

In [46]:
class WithColumns(TransformerMixin):
    def __init__(self, transformer, columns=None):
        self.columns = columns
        self.transformer = transformer
        
    def fit(self, X, y=None, **fit_params):
        self.transformer.fit(X[self.columns])
        return self
    
    def transform(self, X, y=None, **fit_params):
        res = X.copy()
        res[self.columns] = self.transformer.transform(res[self.columns])
        return res

In [47]:
class ReplaceValues(TransformerMixin):
    def __init__(self, mapping):
        self.mapping = mapping
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, y=None, **fit_params):
        res = X.replace(self.mapping)
        return res
# WithColumns(ReplaceValues({
#                                 "VBF": "other",
#                                 "Wp": "other",
#                                 "Z": "other",
#                                 "gg": "other"
#                                }), columns="label").fit_transform(d_test)

In [48]:
class DropByLevel(TransformerMixin):
    def __init__(self, column, threshold=None, limit=None):
        self.column = column
        self.threshold = threshold
        self.limit = limit
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, y=None, **fit_params):
        mask = mask_by_levels(X[self.column], threshold=self.threshold, limit=self.limit)
        
        if np.all(mask):
            return X.copy()
        
        return X[mask].copy()

In [49]:
class DropByValues(TransformerMixin):
    def __init__(self, column, values):
        self.column = column
        self.values = values
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, y=None, **fit_params):
        mask = ~np.any([X[self.column] == v for v in self.values], axis=0)
        
        if np.all(mask):
            return X.copy()
        
        return X[mask].copy()
# DropByValues("label", ["tt"]).fit_transform(d_train)

Feature processing for next to baseline model, we name it `significant imbalanced model` is a bit more complex. As before, we fill empty values for chosen features with zeros.

We also drop `tt`, `Wp` and `Z` events for this step. We try to focus on events that have similar weight per event. `Wp` and `Z` are few but they are very heavy, while `tt` are quite a lot but weight is tiny. It is quite hard to handle such a mixture together.

Now we end up with `gg` and `VBF` events. The dataset is still imbalanced from the number of events perspective, but at the same time we can bother less about their weights.

As a part of feature processing, we also drop negative weights. That's just a technical issue, because the algorithm we are going to apply can't handle negative weights.

In [92]:
SignificantFeaturePipeline = \
    pipeline_maker(Pipeline, steps=[
        ('keep_features', KeepFeatures(subcolumns + ["label", "weight"])),
        ('filna', FillNa(0.)),
        ('drop_tt', DropByValues("label", ["tt", "Wp", "Z"])),
        ('drop_neg_weight', DropByLevel(["weight"], threshold=0)),
        ('labels', WithColumns(LabelEncoder(), columns="label"))
    ])
significant_feature_pipeline = SignificantFeaturePipeline().fit(d_train)
significant_features = significant_feature_pipeline.transform(d_train)
significant_features

,photon_1lead_pt,photon_1lead_E,met_et,photon_2lead_pt,photon_2lead_eta,h_mass,photon_1lead_eta,photon_2lead_E,photon_2lead_etcone20,photon_1lead_phi,photon_2lead_phi,met_phi,photon_1lead_etcone20,label,weight
34762,68985.805,85216.230,30061.690,51873.760,-0.020217,118622.77,0.673179,51884.363,-54.641132,-2.755656,0.261481,-0.911852,-33.08661,1,6.426950e-08
47589,66244.650,68949.734,9151.307,49499.246,-0.664182,129391.59,0.284815,60824.562,-703.480700,-0.899317,2.492585,0.882567,-430.34076,1,6.509352e-08
44652,62168.310,103743.570,81357.300,43533.900,0.573396,133043.72,-1.100176,50888.734,-355.123320,-2.316379,2.214298,-0.120469,-257.24194,1,6.574681e-08
29709,56140.140,92557.170,59095.723,49639.758,0.142135,134930.78,-1.085008,50142.023,-722.904050,1.459849,-2.128604,-0.804518,-515.87780,1,6.081813e-08
11376,56304.918,69706.420,5357.768,51354.156,-0.455490,118769.27,0.676951,56774.160,-774.800000,2.270618,-0.818975,2.244862,-278.63430,1,6.155636e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32745,123812.510,124740.280,55756.870,33976.465,-0.132365,130749.42,0.122344,34274.540,-307.793000,1.774965,-1.117429,-2.609147,-236.87490,0,3.267510e-09
25881,73171.550,77066.600,44342.586,69011.670,0.984790,115833.22,0.324857,105269.270,-502.582800,1.761882,-2.675597,-0.624818,-667.40283,0,1.104680e-08
19695,62460.710,67644.390,74117.960,48761.650,-0.666147,126094.12,0.404643,59986.727,-996.092960,-2.093862,1.519702,0.057860,-138.29385,1,3.565212e-08
40489,66830.560,67717.870,17952.191,45487.150,-0.942891,134209.08,0.162774,67250.305,-70.243450,-1.102934,1.962060,-1.235665,-623.46350,1,6.799428e-08


In [93]:
np.all(significant_features["weight"] > 0)

True

Following the reference about usage of [bagging for imbalanced datasets](https://machinelearningmastery.com/bagging-and-random-forest-for-imbalanced-classification/) we focus on the model of Easy Ensemble. It is an ensemble model, where AdaBoost (a variant of BDT) is used as a base estimator. The goal is to downsample major class in order to balance the trainig sample.

To avoid throwing out the data we train 6 different AdaBoosts on different samples, where minority class is always entirely included, while events from majority class are downsampled to fit number of events in the minority class.

After 6 different base estimators have been trained, the voting procedure decides on the prediction for the specific test event.

In [96]:
class WeightedAdaBoost(AdaBoostClassifier):
    def __init__(self,
                 base_estimator=None, *,
                 n_estimators=50,
                 learning_rate=1.,
                 algorithm='SAMME.R',
                 random_state=None, weights_column=None):

        super().__init__(
            base_estimator=base_estimator,
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            random_state=random_state)
        self.weights_column = weights_column
    
    def fit(self, X, y):
        super().fit(np.delete(X, self.weights_column, 1), y, sample_weight=X[:, self.weights_column])
        
    def predict(self, X):
        return super().predict(np.delete(X, self.weights_column, 1))
    
    def predict_proba(self, X):
        return super().predict_proba(np.delete(X, self.weights_column, 1))

In [94]:
sign_imbal_model = EasyEnsembleClassifier( \
                                          n_estimators=6,
                                          base_estimator=WeightedAdaBoost(weights_column=list(split_targets(significant_features, "label")[0].columns).index("weight")),
                                          random_state=RANDOM_STATE,
                                          sampling_strategy="not minority",
                                          replacement=False
                                         ) \
                   .fit(*split_targets(significant_features, "label"))

We build a report following the same structure as we had for the baseline model.

Here we have 2 classes with a simple confusion matrix. It shows that we still didn't manage to fight the imbalance.

Since here we have ensemble of BDTs, as feature importance we show median of scores per feature.

In [95]:
report(sign_imbal_model, significant_feature_pipeline, significant_feature_pipeline.transform(d_test))

Sum of weights


label
0    0.000019
1    0.000269
Name: weight, dtype: float64

Model classes


array(['VBF', 'gg'], dtype=object)

Confusion matrix, normalized


array([[  56, 1958],
       [  42, 5049]])

-Log confusion matrix of weights


array([[14.35249688, 10.89898206],
       [13.00675829,  8.22907426]])

Feature importances


,score
feature,
photon_1lead_pt,0.21
photon_2lead_eta,0.14
photon_1lead_eta,0.14
photon_2lead_pt,0.09
photon_1lead_E,0.08
met_et,0.08
photon_2lead_E,0.07
h_mass,0.06
met_phi,0.05
